In [49]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import FunctionTransformer

In [50]:
train=pd.read_csv(r'C:\Users\riddh\Downloads\train (1).csv')

In [51]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [52]:
train.shape

(1460, 81)

In [53]:
train.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [54]:
df=pd.DataFrame(data=train.isnull().sum(),columns=['value'])

In [55]:
np.where(k['value']!=0)

(array([ 3,  6, 25, 26, 30, 31, 32, 33, 35, 42, 57, 58, 59, 60, 63, 64, 72,
        73, 74], dtype=int64),)

In [8]:
col=k.index[np.where(k['value']!=0)]

In [10]:
col

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [11]:
train[col].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [56]:
def remove_columns(df,train):
        a=int(60/100*train.shape[0])
        col3=df.index[np.where(df['value']>=a)].tolist()
        train=train.drop(col3,axis=1)
        col2=[]
        for col in train.columns :
            dict_c=train[col].value_counts().to_dict()
            for i in dict_c:
                if(dict_c[i]>=a):
                    col2.append(col)
                    break
        col2.append('Id')
        return (train.drop(col2,axis=1)) 

In [ ]:
train.shape

In [57]:
train=remove_columns(df,train)

In [58]:
train.shape

(1460, 40)

In [ ]:
train.isnull().sum()

In [59]:
train.drop('FireplaceQu',axis=1,inplace=True)

In [17]:
np.where(train.isnull().sum()!=0)

(array([ 1, 11, 12, 14, 15, 29, 30, 31], dtype=int64),)

In [60]:
check_col=train.columns[np.where(train.isnull().sum()!=0)]

In [61]:
impute_num_col=train[check_col].select_dtypes(include='number').columns
impute_cat_col=train[check_col].select_dtypes(exclude='number').columns

In [20]:
impute_num_col

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

In [21]:
impute_cat_col

Index(['MasVnrType', 'BsmtQual', 'BsmtFinType1', 'GarageType', 'GarageFinish'], dtype='object')

# checking imputer

In [62]:
from sklearn.impute import SimpleImputer

In [63]:
numeric_imputer = SimpleImputer(strategy='median')

In [64]:
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [65]:
imputer_transformer=[
    ('numeric',numeric_imputer, impute_num_col),
    ('categorical',categorical_imputer, impute_cat_col)
]


In [66]:
imputer=ColumnTransformer(imputer_transformer)

imputer.fit_transform(train)

In [67]:
imputer.fit_transform(train).shape

(1460, 8)

In [75]:
col=impute_num_col.tolist()+impute_cat_col.tolist()
train[col]=imputer.fit_transform(train)

In [76]:
train.shape

(1460, 39)

In [31]:
# cols=imputed_df.columns.tolist()

In [78]:
num_col=train.select_dtypes(include='number').columns.tolist()
cat_col=train.select_dtypes(exclude='number').columns.tolist()

In [34]:
# num_col.remove('SalePrice')

# Column Transformer

In [79]:
transformer=ColumnTransformer([

#     ("num_col_impute",numeric_imputer,impute_num_col),
#     ("cat_col_impute",categorical_imputer,impute_cat_col),
    ("scaling",StandardScaler(),num_col),
    ("ohe",OneHotEncoder(sparse=False,drop='first'),cat_col)],
remainder='passthrough')

In [37]:
# pipe=Pipeline(steps=[
# #     ('funct_trans',ft),
#     ('transform',transformer),
#                ('svr_model',SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1))])

# pipeline

In [98]:
pipe=Pipeline(steps=[
               ('svr_model',SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1))])

In [38]:
X_train,X_test,y_train,y_test=train_test_split(train.drop('SalePrice',axis=1),train['SalePrice'],random_state=30,test_size=0.2)

In [ ]:
y_train.shape

In [80]:
transformed_train=transformer.fit_transform(train)


In [89]:
train.iloc[:,-1:]

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [94]:
X=transformed_train[:,:-1]
y=transformed_train[:,-1]

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('svr_model', SVR(C=100, gamma=0.1))])

In [103]:
y_pred=pipe.predict(X_test)

In [104]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def evaluation(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

In [105]:
mae, mse, rmse, r_squared = evaluation(y_test, y_pred)

In [106]:
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)

MAE: 0.2821824162776588
MSE: 0.11114757039293059
RMSE: 0.33338801777048105
R2 Score: 0.5550757538975197
